<a href="https://colab.research.google.com/github/bom2542/214347_project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#21434756555555555555
print("Hefjsdl")

Hefjsdl
